## Prepare train, test and side-info

We first need to download and prepare the data files. This can be acomplished using this a built-in function is smurff. IC50 is a compound x protein matrix, The ECFP matrix as features as side information on the compounds.

We also load a notebook extension wurlitzer to make the output that would normally go to the terminal end up in the notebook.

In [ ]:
%load_ext wurlitzer
import smurff

ic50_train, ic50_test, ecfp = smurff.load_chembl()

## Matrix Factorization without Side Information

As a first example we can run SMURFF without side information. The method used here is BPMF.

Input matrix for `Y` is a sparse scipy matrix (either coo_matrix, csr_matrix or csc_matrix). The test matrix
`Ytest` also needs to ne sparse matrix of the same size as `Y`. Here we have used burn-in of 20 samples for the Gibbs sampler and then collected 80 samples from the model. We use 16 latent dimensions in the model.

For good results you will need to run more sampling and burnin iterations (>= 1000) and maybe more latent dimensions.

We create a session, and the `run` method returns the predictions of the `Ytest` matrix. `predictions` is a list of of type `Prediction`.

In [ ]:
session = smurff.BPMFSession(
                       Ytrain     = ic50_train,
                       Ytest      = ic50_test,
                       num_latent = 16,
                       burnin     = 20,
                       nsamples   = 80,
                       verbose    = 0,)

predictions = session.run()
print("First prediction element: ", predictions[0])

rmse = smurff.calc_rmse(predictions)
print("RMSE =", rmse)

## Matrix Factorization with Side Information

If we want to use the compound features we can use the Macau algorithm.

The parameter `side_info = [ecfp, None]` sets the side information for rows and columns, respectively. In this example we only use side information for the compounds (rows of the matrix).


In [ ]:
predictions = smurff.MacauSession(
                       Ytrain     = ic50_train,
                       Ytest      = ic50_test,
                       side_info  = [ecfp, None],
                       num_latent = 16,
                       burnin     = 40,
                       nsamples   = 100).run()

smurff.calc_rmse(predictions)

## Univariate sampler

SMURFF also includes an option to use a `very fast` univariate sampler, i.e., instead of sampling blocks of variables jointly it samples each individually. An example:

In [ ]:
predictions = smurff.MacauSession(
                       Ytrain     = ic50_train,
                       Ytest      = ic50_test,
                       side_info  = [ecfp, None],
                       univariate = True,
                       num_latent = 32,
                       burnin     = 500,
                       nsamples   = 3500)

When using it we recommend using larger values for `burnin` and `nsamples`, because the univariate sampler mixes slower than the blocked sampler.

## Adaptive noise

In the previous examples we fixed the observation noise by specifying `precision = 5.0`. Instead we can also allow the model to automatically determine the precision of the noise by setting signal-to-noise ratio parameters `sn_init` and `sn_max`.

`sn_init` is an initial signal-to-noise ratio.

`sn_max`  is the maximum allowed signal-to-noise ratio. This means that if the updated precision would imply a higher signal-to-noise ratio than `sn_max`, then the precision value is set to `(sn_max + 1.0) / Yvar` where `Yvar` is the variance of the training dataset `Y`.

In [ ]:
result = smurff.smurff(Y          = ic50_train,
                       Ytest      = ic50_test,
                       priors     = ['macauone', 'normal'],
                       side_info  = [ecfp, None],
                       aux_data   = [[], []],
                       num_latent = 32,
                       sn_init    = 0,
                       sn_max     = 1,
                       burnin     = 500,
                       nsamples   = 3500)

## Binary matrices

SMURFF can also factorize binary matrices (with or without side information). As an input the sparse matrix should only contain values of 0 or 1. To factorize them we employ probit noise model that can be enabled by setting `threshold` parameter.

Care has to be taken to make input to the model, as operating with sparse matrices can drop real 0 measurements. In the below example, we first copy the matrix (line 9) and then threshold the data to binary (line 10).

In [ ]:
## using activity threshold pIC50 > 6.5
act = ic50
act.data = act.data > 6.5
act_train, act_test = smurff.make_train_test(act, 0.5)

## running factorization (Macau)
result = smurff.smurff(Y          = act_train,
                       Ytest      = act_test,
                       priors     = ['macau', 'normal'],
                       side_info  = [ecfp, None],
                       aux_data   = [[], []],
                       num_latent = 32,
                       threshold  = 0.5,
                       burnin     = 500,
                       nsamples   = 3500)

## Tensor Factorization

SMURFF also supports tensor factorization with and without side information on any of the modes. Tensor can be thought as generalization of matrix to relations with more than two items. For example 3-tensor of `drug x cell x gene` could express the effect of a drug on the given cell and gene. In this case the prediction for the element `Yhat[i,j,k]`* is given by

$$ \hat{Y}_{ijk} = \sum_{d=1}^{D}u^{(1)}_{d,i}u^{(2)}_{d,j}u^{(3)}_{d,k} + mean $$

Visually the model can be represented as follows:

<img src="https://macau.readthedocs.io/en/latest/_images/tensor-model.png" alt="tesor-model" style="width: 50%; height: 50%"/>
<center><i>Tensor model predicts <strong><code>Yhat[i,j,k]</code></strong> by multiplying all latent vectors together element-wise and then taking the sum along the latent dimension (figure omits the global mean).</i></center>

For tensors SMURFF packages uses Pandas `DataFrame` where each row stores the coordinate and the value of a known cell in the tensor. Specifically, the integer columns in the DataFrame give the coordinate of the cell and `float` (or double) column stores the value in the cell (the order of the columns does not matter). The coordinates are 0-based.

Here is a simple toy example with factorizing a 3-tensor with side information on the first mode.

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse
import smurff
import itertools

## generating toy data
A = np.random.randn(15, 2)
B = np.random.randn(3, 2)
C = np.random.randn(2, 2)

idx = list( itertools.product(np.arange(A.shape[0]),
                              np.arange(B.shape[0]),
                              np.arange(C.shape[0])) )
df  = pd.DataFrame( np.asarray(idx), columns=["A", "B", "C"])
df["value"] = np.array([ np.sum(A[i[0], :] * B[i[1], :] * C[i[2], :]) for i in idx ])

## assigning 20% of the cells to test set
Ytrain, Ytest = smurff.make_train_test_df(df, 0.2)

## for artificial dataset using small values for burnin, nsamples and num_latents is fine
results = smurff.smurff(Ytrain,
                        Ytest=Ytest,
                        priors=['normal', 'normal', 'normal'],
                        side_info=[None, None, None],
                        aux_data=[[], [], []],
                        num_latent=4,
                        precision=50,
                        burnin=20,
                        nsamples=20)